# Advent of Code - 2024 - Day 17 - Problem 2

https://adventofcode.com/2024/day/17

## Load Source Data

Load the map data into `DATA`.

In [2]:
f = open("data/day17.txt", "r")
DATA = list(map(str.strip, f.readlines()))
f.close()

# DATA = """Register A: 729
# Register B: 0
# Register C: 0

# Program: 0,1,5,4,3,0"""
# DATA = list(map(str.strip, DATA.splitlines()))

# DATA

## Create Emulator Class

In [3]:
class Emulator:

    # Instructions
    ADV = 0
    BDV = 6
    BST = 2
    BXC = 4
    BXL = 1
    CDV = 7
    JNZ = 3
    OUT = 5

    # Registers
    A = 0
    B = 1
    C = 2

    def __init__(self):
        self._registers = [0, 0, 0]
        self._ip = 0
        self._code = []
        self._output = []
        self._trace = False

    def load_program(self, program):
        for register in range(3):
            line = program[register]
            idx = line.index(":")
            value = int(line[idx + 1 :])
            self._registers[register] = value

        line = program[4]
        idx = line.index(":")
        self._code = list(map(int, line[idx + 1 :].split(",")))

    def execute(self):
        opcode = self._get_opcode()
        if opcode == None:
            return False

        if opcode == self.ADV:
            self._execute_adv()
        elif opcode == self.BDV:
            self._execute_bdv()
        elif opcode == self.BST:
            self._execute_bst()
        elif opcode == self.BXC:
            self._execute_bxc()
        elif opcode == self.BXL:
            self._execute_bxl()
        elif opcode == self.CDV:
            self._execute_cdv()
        elif opcode == self.JNZ:
            self._execute_jnz()
        elif opcode == self.OUT:
            self._execute_out()

        return True

    def get_output(self):
        return self._output

    def set_trace(self, trace):
        self._trace = trace

    #
    # State Management
    #

    def _advance_ip(self):
        self._ip += 2

    def _set_ip(self, ip):
        self._ip = ip

    def _get_opcode(self):
        if self._ip >= len(self._code):
            return None
        return self._code[self._ip]

    def _get_register(self, register):
        return self._registers[register]

    def _set_register(self, register, value):
        self._registers[register] = value

    def _write_output(self, value):
        self._output.append(value)

    def _get_literal_operand(self):
        return self._code[self._ip + 1]

    def _get_combo_operand(self):
        value = self._code[self._ip + 1]
        if value <= 3:
            return value
        elif value == 4:
            return self._get_register(self.A)
        elif value == 5:
            return self._get_register(self.B)
        elif value == 6:
            return self._get_register(self.C)
        else:
            raise Exception(f"Illegal value {value}")

    def _get_combo_description(self):
        value = self._code[self._ip + 1]
        if value <= 3:
            return str(value)
        elif value == 4:
            return "A"
        elif value == 5:
            return "B"
        elif value == 6:
            return "C"
        else:
            raise Exception(f"Illegal value {value}")

    #
    # Instructions
    #

    def _execute_adv(self):
        register = self._get_register(self.A)
        operand = self._get_combo_operand()
        result = register // pow(2, operand)

        if self._trace:
            print(f"{self._ip}:ADV - A({oct(result)}) = A({oct(register)}) / 2 ** {self._get_combo_description()}({oct(operand)})")

        self._set_register(self.A, result)
        self._advance_ip()

    def _execute_bdv(self):
        register = self._get_register(self.A)
        operand = self._get_combo_operand()
        result = register // pow(2, operand)

        if self._trace:
            print(f"{self._ip}:BDV - B({oct(result)}) = A({oct(register)}) / 2 ** {self._get_combo_description()}({oct(operand)})")

        self._set_register(self.B, result)
        self._advance_ip()

    def _execute_bst(self):
        operand = self._get_combo_operand()
        result = operand % 8

        if self._trace:
            print(f"{self._ip}:BST - B({oct(result)}) = {self._get_combo_description()}({oct(operand)}) % 8")

        self._set_register(self.B, result)
        self._advance_ip()

    def _execute_bxc(self):
        register_b = self._get_register(self.B)
        register_c = self._get_register(self.C)
        result = register_b ^ register_c

        if self._trace:
            print(f"{self._ip}:BXC - B({oct(result)}) = B({oct(register_b)}) ^ C({oct(register_c)})")

        self._set_register(self.B, result)
        self._advance_ip()

    def _execute_bxl(self):
        register_b = self._get_register(self.B)
        value = self._get_literal_operand()
        result = register_b ^ value

        if self._trace:
            print(f"{self._ip}:BXL - B({oct(result)}) = B({oct(register_b)}) ^ {oct(value)}")

        self._set_register(self.B, result)
        self._advance_ip()

    def _execute_cdv(self):
        register = self._get_register(self.A)
        operand = self._get_combo_operand()
        result = register // pow(2, operand)

        if self._trace:
            print(f"{self._ip}:CDV - C({oct(result)}) = A({oct(register)}) / 2 ** {self._get_combo_description()}({oct(operand)})")

        self._set_register(self.C, result)
        self._advance_ip()

    def _execute_jnz(self):
        register = self._get_register(self.A)
        value = self._get_literal_operand()

        if self._trace:
            print(f"{self._ip}:JNZ - A({oct(register)}) -> {value}")

        if register == 0:
            self._advance_ip()
        else:
            self._set_ip(value)

    def _execute_out(self):
        value = self._get_combo_operand()
        result = value % 8

        if self._trace:
            print(f"{self._ip}:OUT - {oct(result)} = {self._get_combo_description()}({oct(value)}) % 8")

        self._write_output(result)
        self._advance_ip()

## Define find_input

In [79]:
def find_input(starting_with, producing):
    for digit in range(8):
        input = starting_with + str(digit)
        emulator = Emulator()
        emulator.load_program(DATA)
        emulator._set_register(emulator.A,int(input,8) )
        while emulator.execute():
            pass
        result = "".join(map(str, emulator.get_output()))
        if result == producing:
            yield input


# 0355544130571142
def find_input_2(producing):
    if producing == "": yield ""
    else:
        for starting_with in find_input_2(producing[1:]):
            print(f"starting_width = {starting_with} for {producing[1:]}")
            for input in find_input(starting_with, producing):
                yield input

# for result in find_input("561", "5530"):
#     print(result)

for result in find_input_2("2411750314455530"):
    print(result)

starting_width =  for 
starting_width = 5 for 0
starting_width = 56 for 30
starting_width = 560 for 530
starting_width = 5600 for 5530
starting_width = 56000 for 55530
starting_width = 56001 for 55530
starting_width = 560013 for 455530
starting_width = 5600132 for 4455530
starting_width = 56001327 for 14455530
starting_width = 560013275 for 314455530
starting_width = 5600132756 for 0314455530
starting_width = 56001327560 for 50314455530
starting_width = 560013275602 for 750314455530
starting_width = 5600132756024 for 1750314455530
starting_width = 5600132756025 for 1750314455530
starting_width = 56001327560250 for 11750314455530
starting_width = 560013275602505 for 411750314455530
5600132756025052
5600132756025055
5600132756025057
starting_width = 56001327561 for 50314455530
starting_width = 56001327566 for 50314455530
starting_width = 5601 for 5530
starting_width = 56011 for 55530
starting_width = 560113 for 455530
starting_width = 5601132 for 4455530
starting_width = 56011327 for 144

## Determine Octal Strings

In [43]:
print(oct(int("7777777",8)))
digit_0 = ["2", "3", "4", "5", "6", "7"]
digit_1 = ["0", "2", "4", "5", "6", "7"]
digit_2 = ["2", "3", "4", "5", "6", "7"]
digit_3 = ["2", "4", "5", "6", "7"]
digit_4 = ["2", "3", "4", "5", "6", "7"]
digit_5 = ["0", "1", "2", "4", "5", "6", "7"]
digit_6 = ["2", "3", "4", "5", "6", "7"]
digit_7 = ["2", "4", "5", "6", "7"]

# for o1 in digit_2:
#     for o2 in digit_4:
#         for o3 in digit_1:
#             for o4 in digit_1:
#                 for o5 in digit_7:
#                     for o6 in digit_5:
#                         for o7 in digit_0:
#                             for o8 in digit_3:
#                                 for o9 in digit_1:
#                                     o_string = o9+o8+o7+o6 + o5 + o4 + o3 + o2 + o1
#                                     a = int(o_string, 8)
#                                     # print(a)

#                                     emulator = Emulator()
#                                     emulator.load_program(DATA)
#                                     emulator._set_register(emulator.A, a)
#                                     while emulator.execute():
#                                         pass

#                                     output = emulator.get_output()
#                                     try:
#                                         if (
#                                             output[0] == 2
#                                             and output[1] == 4
#                                             and output[2] == 1
#                                                 and output[3] == 1
#                                                 and output[4] == 7
#                                                 # and output[5] == 5
#                                             ):
#                                             print(oct(a) + " = " + str(output))
#                                     except IndexError:
#                                         print(o_string)
#                                         print(oct(a) + " = " + str(output))

last_digit = set()
for a in range(int("7", 8)):
    emulator = Emulator()
    emulator.load_program(DATA)
    emulator._set_register(emulator.A,a )
    while emulator.execute():
        pass
    if emulator.get_output()[0] == 0:
        # print(oct(a))
        last_digit.add((oct(a)[-1]))

last_digit

0o7777777


{'5'}

## Run Program

Parses the input `DATA` into `MAP`.

In [81]:
a = int("5600132756025052", 8)
print(a)
emulator = Emulator()
emulator.load_program(DATA)
emulator._set_register(emulator.A, a)
#emulator.set_trace(True)

while emulator.execute():
    pass

print(emulator.get_output())

# #offsets = [3, 2, 524283]
# offsets = [1]
# offset_idx = 0
# last_offset = 0
# #for offset in range(3156527, 500000000, 7168):
# #for offset in range(149648058927, 5000000000000, 117440512):
# #for offset in range(149648058927-(240518168576)*7, 500000000000000, 240518168576//32):
# #for offset in range(0, 100000, 1):
# offset = 10794
# while offset < 200000:
# #for offset in range(0, 20000000000):

#     # for offset in range(8):
#     emulator = Emulator()
#     emulator.load_program(DATA)
#     emulator._set_register(emulator.A, MIN + offset)

#     while emulator.execute():
#         pass

#     if ( emulator.get_output()[0] == 2 
#         #emulator.get_output()[1] == 4 and
#         #emulator.get_output()[2] == 1 and
#         #emulator.get_output()[3] == 1 and
#         #emulator.get_output()[4] == 7 and
#         #emulator.get_output()[5] == 5 and
#         #emulator.get_output()[6] == 0 and
#         #emulator.get_output()[7] == 3 and
#         #emulator.get_output()[8] == 1 and
#         #emulator.get_output()[9] == 4 and
#         #emulator.get_output()[10] == 4 
#         #emulator.get_output()[11] == 5 
#         # emulator.get_output()[12] == 5 and
#         # emulator.get_output()[13] == 5 and
#         # emulator.get_output()[14] == 3 and
#         #emulator.get_output()[15] == 0
#         ):
#         print(f"{offset}, {offset - last_offset}, " + "-".join(map(str, emulator.get_output())) + f", {len(emulator.get_output())}")
#         last_offset = offset

#     offset += offsets[offset_idx]
#     offset_idx = (offset_idx+1) % len(offsets)

202322348616234
[2, 4, 1, 1, 7, 5, 0, 3, 1, 4, 4, 5, 5, 5, 3, 0]
